In [1]:
from TwitterAPI import TwitterAPI
import numpy as np
import pandas as pd
from datetime import datetime

# Functions

In [2]:
def get_twitter():
    """ 
    Generates twitter connection
    """
    
    #Tokens
    consumer_key='nDx2bQS2DVdXEZd0b8MwaEG2L'
    consumer_secret='BeA1VuBq7hyi01fo4LFD13YAcrDgeYxIJTQgzx4gg8H5fSc3FR'
    access_token='900568210424164352-pKLEmPfNcqyGi3q67pyK3bJYtc5G01S'
    access_token_secret='uGbegX9whUY0FjZS6tgIrDITVOM8Q6tKMODNVPG2sT9Aq'
    
    
    twitter = TwitterAPI(
                   consumer_key,
                   consumer_secret,
                   access_token,
                   access_token_secret)
    return twitter

def get_NumberofFollowers(screen_name):       #does not crash when Exception, but does not correct exceptions either.
    '''
    returns the number of friends the user has
    '''
    errorOccurred = False
    try:
        request=twitter.request('users/lookup',{'screen_name':screen_name})
        return request.json()[0]['followers_count']
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        errorOccurred = True
        return np.NaN    

# Import Acsounts

In [3]:
accounts_df = pd.read_csv('Data/accounts.csv',sep=";")

# Collect Data

In [4]:
twitter = get_twitter()

In [5]:
accounts = accounts_df['Account'] #pd series
todaysFollowers = pd.DataFrame(accounts, columns=['Account'])
date=str(datetime.now().date())
# date = '2018-02-13' #for when I execute it after midnight
todaysFollowers[date] = np.NaN

In [6]:
todaysFollowers[date] = todaysFollowers['Account'].apply(get_NumberofFollowers)

In [ ]:
todaysFollowers.head()

In [ ]:
todaysFollowers.loc[pd.isnull(todaysFollowers[date])] #check nulls

# Save Data

In [ ]:
todaysFollowers.to_csv('Data/todaysFollowers_'+date+'.csv',sep=';',index=False)

# Add to Historic Data

In [ ]:
historicData = pd.read_csv('Data/todaysFollowers_all.csv',sep = ';') #load historic

In [ ]:
historicData = pd.concat([historicData,todaysFollowers[date]],axis=1) #add todays column

In [ ]:
historicData.head()

In [ ]:
historicData.to_csv('Data/todaysFollowers_all.csv',sep=';',index=False) #save